# Tutorial 3: SQL Analytics in Databricks

## Overview
This notebook covers SQL operations in Databricks:
- SQL Fundamentals with Spark SQL
- Advanced SQL techniques (Window functions, CTEs)
- Data warehousing concepts (Tables, Views, Optimization)

**Key Concepts:**
- Spark SQL: Query DataFrames using SQL syntax
- Temp Views: Create temporary SQL-queryable tables
- Persistent Tables: Store data in Delta Lake format
- Query Optimization: Improve query performance

In [0]:
# Import libraries
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Load datasets
customer_df = spark.read.csv("/Volumes/workspace/sample/datasets/customer_data.csv", header=True, inferSchema=True)
products_df = spark.read.csv("/Volumes/workspace/sample/datasets/products.csv", header=True, inferSchema=True)
sales_df = spark.read.csv("/Volumes/workspace/sample/datasets/sales_data.csv", header=True, inferSchema=True)


print("Datasets loaded successfully!")

## 1. SQL Fundamentals

### Creating Temporary Views

**Concept:** Temporary views allow you to query DataFrames using SQL

**Syntax:**
- `.createOrReplaceTempView("view_name")`: Session-scoped view
- `.createOrReplaceGlobalTempView("view_name")`: Global view (access with `global_temp.view_name`)

In [0]:
# Create temporary views for SQL queries
customer_df.createOrReplaceTempView("customers")
products_df.createOrReplaceTempView("products")
sales_df.createOrReplaceTempView("sales")


print("Temporary views created: customers, products, sales, web_traffic")

### Basic SELECT Queries

**SQL Syntax:**
```sql
SELECT column1, column2
FROM table_name
WHERE condition
ORDER BY column
LIMIT n
```

In [0]:
%sql
-- Simple SELECT: View first 10 customers
SELECT customer_id, first_name, last_name, email, city, state
FROM customers
LIMIT 10

In [0]:
%sql
-- SELECT with WHERE clause: Filter customers by state
SELECT customer_id, first_name, last_name, city, annual_income
FROM customers
WHERE state = 'California'
ORDER BY annual_income DESC
LIMIT 10

In [0]:
%sql
-- SELECT with multiple conditions
SELECT customer_id, first_name, last_name, age, annual_income, segment
FROM customers
WHERE age > 30 
  AND annual_income > 50000
  AND email_subscribed = true
ORDER BY annual_income DESC
LIMIT 20

### Aggregate Functions

**Common Aggregations:**
- `COUNT()`: Count rows
- `SUM()`: Sum values
- `AVG()`: Average values
- `MIN()` / `MAX()`: Minimum / Maximum values
- `STDDEV()`: Standard deviation

In [0]:
%sql
-- Aggregate functions: Customer statistics
SELECT 
  COUNT(*) as total_customers,
  AVG(age) as avg_age,
  AVG(annual_income) as avg_income,
  MIN(annual_income) as min_income,
  MAX(annual_income) as max_income,
  STDDEV(annual_income) as stddev_income
FROM customers

### GROUP BY

**Purpose:** Aggregate data by categories

**Syntax:**
```sql
SELECT column, AGG_FUNCTION(column)
FROM table
GROUP BY column
HAVING condition
```

In [0]:
%sql
-- GROUP BY: Customer distribution by segment
SELECT 
  segment,
  COUNT(*) as customer_count,
  AVG(age) as avg_age,
  AVG(annual_income) as avg_income,
  SUM(CASE WHEN email_subscribed THEN 1 ELSE 0 END) as subscribed_count
FROM customers
GROUP BY segment
ORDER BY customer_count DESC

In [0]:
%sql
-- GROUP BY with HAVING: States with high-income customers
SELECT 
  state,
  COUNT(*) as customer_count,
  AVG(annual_income) as avg_income
FROM customers
GROUP BY state
HAVING AVG(annual_income) > 60000
ORDER BY avg_income DESC
LIMIT 10

### JOINs

**Types of Joins:**
- `INNER JOIN`: Returns matching rows from both tables
- `LEFT JOIN`: All rows from left table + matching from right
- `RIGHT JOIN`: All rows from right table + matching from left
- `FULL OUTER JOIN`: All rows from both tables

In [0]:
%sql
-- Note: Since our sales table has product names (not IDs), 
-- we'll create a simulated join scenario

-- Sales by region with aggregated metrics
SELECT 
  s.region,
  COUNT(DISTINCT s.transaction_id) as total_transactions,
  COUNT(DISTINCT s.product) as unique_products,
  SUM(s.total_sales) as total_revenue,
  AVG(s.customer_satisfaction) as avg_satisfaction,
  SUM(s.quantity) as total_units_sold
FROM sales s
GROUP BY s.region
ORDER BY total_revenue DESC

In [0]:
%sql
-- Product analysis with category grouping
SELECT 
  category,
  COUNT(*) as product_count,
  AVG(price) as avg_price,
  AVG(rating) as avg_rating,
  SUM(num_reviews) as total_reviews,
  MIN(price) as min_price,
  MAX(price) as max_price
FROM products
GROUP BY category
ORDER BY avg_rating DESC

## 2. Advanced SQL Techniques

### Window Functions

**Purpose:** Perform calculations across rows related to current row

**Common Window Functions:**
- `ROW_NUMBER()`: Assigns unique row number
- `RANK()` / `DENSE_RANK()`: Ranking with or without gaps
- `LAG()` / `LEAD()`: Access previous/next row values
- `SUM()`, `AVG()` with OVER: Running totals/averages

In [0]:
%sql
-- ROW_NUMBER: Rank customers by income within each segment
SELECT 
  customer_id,
  first_name,
  last_name,
  segment,
  annual_income,
  ROW_NUMBER() OVER (PARTITION BY segment ORDER BY annual_income DESC) as income_rank
FROM customers
QUALIFY income_rank <= 5
ORDER BY segment, income_rank

In [0]:
%sql
-- Running total: Cumulative sales over time
WITH daily_sales AS (
  SELECT 
    date,
    SUM(total_sales) as daily_revenue
  FROM sales
  GROUP BY date
  ORDER BY date
)
SELECT 
  date,
  daily_revenue,
  SUM(daily_revenue) OVER (ORDER BY date) as cumulative_revenue,
  AVG(daily_revenue) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as moving_avg_7days
FROM daily_sales
ORDER BY date

In [0]:
%sql
-- LAG function: Compare with previous period
WITH daily_metrics AS (
  SELECT 
    date,
    SUM(total_sales) as daily_revenue,
    COUNT(transaction_id) as transaction_count
  FROM sales
  GROUP BY date
)
SELECT 
  date,
  daily_revenue,
  LAG(daily_revenue, 1) OVER (ORDER BY date) as prev_day_revenue,
  daily_revenue - LAG(daily_revenue, 1) OVER (ORDER BY date) as revenue_change,
  ROUND(((daily_revenue - LAG(daily_revenue, 1) OVER (ORDER BY date)) / 
         LAG(daily_revenue, 1) OVER (ORDER BY date) * 100), 2) as pct_change
FROM daily_metrics
ORDER BY date DESC
LIMIT 30

### Common Table Expressions (CTEs)

**Purpose:** Create temporary named result sets for complex queries

**Syntax:**
```sql
WITH cte_name AS (
  SELECT ...
)
SELECT * FROM cte_name
```

**Benefits:**
- Improves query readability
- Can reference multiple times
- Useful for breaking down complex logic

In [0]:
%sql
-- CTE Example: Multi-step analysis
WITH customer_stats AS (
  SELECT 
    segment,
    COUNT(*) as customer_count,
    AVG(annual_income) as avg_income,
    AVG(age) as avg_age
  FROM customers
  GROUP BY segment
),
segment_ranking AS (
  SELECT 
    segment,
    customer_count,
    avg_income,
    avg_age,
    RANK() OVER (ORDER BY customer_count DESC) as size_rank,
    RANK() OVER (ORDER BY avg_income DESC) as income_rank
  FROM customer_stats
)
SELECT 
  segment,
  customer_count,
  ROUND(avg_income, 2) as avg_income,
  ROUND(avg_age, 1) as avg_age,
  size_rank,
  income_rank
FROM segment_ranking
ORDER BY size_rank

In [0]:
%sql
-- Complex CTE: Product performance analysis
WITH product_metrics AS (
  SELECT 
    category,
    brand,
    COUNT(*) as product_count,
    AVG(price) as avg_price,
    AVG(rating) as avg_rating,
    SUM(num_reviews) as total_reviews
  FROM products
  WHERE NOT discontinued
  GROUP BY category, brand
),
category_summary AS (
  SELECT 
    category,
    AVG(avg_price) as category_avg_price,
    AVG(avg_rating) as category_avg_rating
  FROM product_metrics
  GROUP BY category
)
SELECT 
  pm.category,
  pm.brand,
  pm.product_count,
  ROUND(pm.avg_price, 2) as brand_avg_price,
  ROUND(cs.category_avg_price, 2) as category_avg_price,
  ROUND(pm.avg_rating, 2) as brand_rating,
  ROUND(cs.category_avg_rating, 2) as category_rating,
  pm.total_reviews
FROM product_metrics pm
JOIN category_summary cs ON pm.category = cs.category
ORDER BY pm.category, pm.product_count DESC

### Subqueries

**Types:**
- Scalar subquery: Returns single value
- Row subquery: Returns single row
- Table subquery: Returns multiple rows/columns

In [0]:
%sql
-- Subquery: Find customers with above-average income
SELECT 
  customer_id,
  first_name,
  last_name,
  annual_income,
  segment
FROM customers
WHERE annual_income > (SELECT AVG(annual_income) FROM customers)
ORDER BY annual_income DESC
LIMIT 20

In [0]:
%sql
-- Correlated subquery: Find top products by category
SELECT 
  p1.category,
  p1.product_name,
  p1.price,
  p1.rating
FROM products p1
WHERE p1.rating >= (
  SELECT AVG(p2.rating)
  FROM products p2
  WHERE p2.category = p1.category
)
ORDER BY p1.category, p1.rating DESC

## 3. Data Warehousing Concepts

### Creating Tables

**Table Types in Databricks:**
- **Managed Tables**: Databricks manages both metadata and data
- **External Tables**: Only metadata managed, data stored externally
- **Delta Tables**: Optimized format with ACID transactions

In [0]:
%sql
-- Create a managed table from query results
CREATE OR REPLACE TABLE customer_segments AS
SELECT 
  segment,
  COUNT(*) as customer_count,
  AVG(age) as avg_age,
  AVG(annual_income) as avg_income,
  MIN(annual_income) as min_income,
  MAX(annual_income) as max_income,
  STDDEV(annual_income) as stddev_income
FROM customers
GROUP BY segment

In [0]:
%sql
-- Query the newly created table
SELECT * FROM customer_segments
ORDER BY customer_count DESC

In [0]:
%sql
-- Create table with explicit schema
CREATE OR REPLACE TABLE sales_summary (
  region STRING,
  total_transactions INT,
  total_revenue DOUBLE,
  avg_satisfaction DOUBLE,
  total_quantity INT
)
USING DELTA;

-- Insert data into the table
INSERT INTO sales_summary
SELECT 
  region,
  COUNT(transaction_id) as total_transactions,
  SUM(total_sales) as total_revenue,
  AVG(customer_satisfaction) as avg_satisfaction,
  SUM(quantity) as total_quantity
FROM sales
GROUP BY region

In [0]:
%sql
SELECT * FROM sales_summary
ORDER BY total_revenue DESC

### Creating Views

**Views vs Tables:**
- Views are virtual tables (don't store data)
- Always reflect current data from source tables
- Useful for simplifying complex queries
- Can control access to sensitive data

In [0]:
%sql
-- Create a temporary view for high-value customers
CREATE OR REPLACE TEMPORARY VIEW high_value_customers AS
SELECT 
  customer_id,
  first_name,
  last_name,
  email,
  city,
  state,
  segment,
  annual_income,
  age
FROM customers
WHERE annual_income > 75000
  AND email_subscribed = true

In [0]:
%sql
-- Query the view
SELECT segment, COUNT(*) as count, AVG(annual_income) as avg_income
FROM high_value_customers
GROUP BY segment
ORDER BY count DESC

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW product_performance AS
SELECT 
  category,
  product_name,
  brand,
  price,
  rating,
  num_reviews,
  price * num_reviews as revenue_potential,
  CASE 
    WHEN rating >= 4.5 THEN 'Excellent'
    WHEN rating >= 4.0 THEN 'Good'
    WHEN rating >= 3.0 THEN 'Average'
    ELSE 'Poor'
  END as rating_category,
  CASE
    WHEN price < 50 THEN 'Budget'
    WHEN price < 100 THEN 'Mid-Range'
    ELSE 'Premium'
  END as price_tier
FROM products
WHERE NOT discontinued

In [0]:
%sql
-- Analyze using the view
SELECT 
  category,
  rating_category,
  COUNT(*) as product_count,
  AVG(price) as avg_price
FROM product_performance
GROUP BY category, rating_category
ORDER BY category, rating_category

### Query Optimization

**Optimization Techniques:**
1. **Partitioning**: Organize data by column values
2. **Caching**: Store frequently accessed data in memory
3. **Predicate Pushdown**: Filter data early
4. **Column Pruning**: Select only needed columns
5. **Broadcast Joins**: For small tables

In [0]:
%sql
-- Use EXPLAIN to see query execution plan
EXPLAIN FORMATTED
SELECT 
  c.segment,
  COUNT(*) as customer_count,
  AVG(c.annual_income) as avg_income
FROM customers c
WHERE c.state IN ('California', 'Texas', 'New York')
GROUP BY c.segment

In [0]:
# Remove caching, just trigger an action to materialize the DataFrame
count = customer_df.count()
print(f"Processed {count} customer records")

### Advanced Aggregations

**CUBE and ROLLUP:**
- `ROLLUP`: Creates subtotals and grand totals (hierarchical)
- `CUBE`: Creates all possible aggregation combinations
- `GROUPING SETS`: Specify exact grouping combinations

In [0]:
%sql
-- ROLLUP: Hierarchical aggregation
SELECT 
  state,
  segment,
  COUNT(*) as customer_count,
  AVG(annual_income) as avg_income
FROM customers
WHERE state IN ('California', 'Texas', 'New York', 'Florida')
GROUP BY ROLLUP (state, segment)
ORDER BY state, segment

In [0]:
%sql
-- CUBE: All combinations
SELECT 
  segment,
  CASE 
    WHEN age < 30 THEN 'Young'
    WHEN age < 50 THEN 'Middle'
    ELSE 'Senior'
  END as age_group,
  COUNT(*) as customer_count,
  AVG(annual_income) as avg_income
FROM customers
GROUP BY CUBE (segment, age_group)
ORDER BY segment, age_group

### PIVOT Operations

**Purpose:** Transform rows to columns

In [0]:
%sql
-- PIVOT: Customer segments by state
SELECT * FROM (
  SELECT state, segment, customer_id
  FROM customers
  WHERE state IN ('California', 'Texas', 'New York', 'Florida', 'Illinois')
)
PIVOT (
  COUNT(customer_id)
  FOR segment IN ('Premium', 'Standard', 'Basic', 'VIP')
)
ORDER BY state

## 4. Performance Best Practices

### Tips for Writing Efficient Queries:

1. **Filter Early**: Use WHERE clauses to reduce data volume
2. **Select Only Needed Columns**: Avoid SELECT *
3. **Use Appropriate Joins**: Choose the right join type
4. **Leverage Partitioning**: Query on partitioned columns
5. **Cache Frequently Used Data**: Use .cache() for repeated access
6. **Use Delta Tables**: Better performance and features
7. **Optimize Window Functions**: Limit partition size when possible
8. **Avoid Cartesian Joins**: Always include join conditions

In [0]:
# Example of optimized query structure

# BAD: Select everything, filter late
# SELECT * FROM large_table WHERE condition

# GOOD: Select specific columns, filter early
# SELECT col1, col2, col3 FROM large_table WHERE condition

# Demonstrate with explain
spark.sql("""
  SELECT customer_id, first_name, last_name, annual_income
  FROM customers
  WHERE state = 'California' 
    AND annual_income > 50000
  LIMIT 100
""").explain()

## Key Takeaways

**SQL Fundamentals:**
- Use temporary views to query DataFrames with SQL
- Master SELECT, WHERE, GROUP BY, and JOIN operations
- Understand aggregate functions and their use cases

**Advanced SQL:**
- Window functions enable powerful row-based calculations
- CTEs improve query readability and maintainability
- Subqueries provide flexible data filtering

**Data Warehousing:**
- Create tables and views for persistent storage
- Use Delta Lake format for ACID transactions
- Optimize queries with caching, partitioning, and proper indexing

**Performance:**
- Filter early, select specific columns
- Use EXPLAIN to understand query plans
- Cache frequently accessed data

**Next Steps:** Move to Notebook 4 for ETL & Data Processing!